In [8]:
import setup
setup.init_django()

In [9]:
import helpers.clients as helper_clients

In [38]:
company_name = "Google"
company_ticker = "GOOG"
multiplier = 1
from_date = "2023-01-09"
to_date = "2023-04-09"

In [39]:
client = helper_clients.PolygonAPIClient(
    ticker=company_ticker,
    multiplier=multiplier,
    from_date=from_date,
    to_date=to_date
)
dataset = client.get_stock_data()

In [40]:
len(dataset)

5000

In [41]:
new_stock_data = dataset[0]
new_stock_data

{'open_price': 88.08,
 'close_price': 88.16,
 'high_price': 88.17,
 'low_price': 88.08,
 'number_of_trades': 55,
 'volume': 1684,
 'volume_weighted_average': 88.1217,
 'time': datetime.datetime(2023, 1, 9, 9, 0, tzinfo=<UTC>)}

In [42]:
from market.models import Company, StockQuote

In [43]:
company_obj, created = Company.objects.get_or_create(name=company_name, ticker=company_ticker)

In [44]:
company_obj

<Company: Company object (2)>

In [45]:
# StockQuote.objects.bulk_create(company=company_obj, **new_stock_data)

In [46]:
# new_quotes = []

# for data in dataset:
#     new_quotes.append(
#         StockQuote(company=company_obj, **data)
#     )

In [47]:
StockQuote.objects.all()

<QuerySet [<StockQuote: StockQuote object (1)>, <StockQuote: StockQuote object (4)>, <StockQuote: StockQuote object (5)>, <StockQuote: StockQuote object (6)>, <StockQuote: StockQuote object (7)>, <StockQuote: StockQuote object (8)>, <StockQuote: StockQuote object (9)>, <StockQuote: StockQuote object (10)>, <StockQuote: StockQuote object (11)>, <StockQuote: StockQuote object (12)>, <StockQuote: StockQuote object (13)>, <StockQuote: StockQuote object (14)>, <StockQuote: StockQuote object (15)>, <StockQuote: StockQuote object (16)>, <StockQuote: StockQuote object (17)>, <StockQuote: StockQuote object (18)>, <StockQuote: StockQuote object (19)>, <StockQuote: StockQuote object (20)>, <StockQuote: StockQuote object (21)>, <StockQuote: StockQuote object (22)>, '...(remaining elements truncated)...']>

In [48]:
batch_size = 1000
for i in range(0, len(dataset), batch_size):
    batch_chunk = dataset[i:i+batch_size]
    chunked_quotes = []
    for data in batch_chunk:
        chunked_quotes.append(
            StockQuote(company=company_obj, **data)
        )
    StockQuote.objects.bulk_create(chunked_quotes, ignore_conflicts=True)

In [49]:
StockQuote.objects.all().count()

10000